<a href="https://colab.research.google.com/github/armanali0786/armanali0786/blob/main/tree_leaves_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Importing Numpy Libraries
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
# import torch.utils.data as data
# from torch.utils.data import Dataset
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
if __name__ == '__main__':
    # data_dir = "/content/drive/My Drive/Cat_Dog_Imageset"
    # train_dir = data_dir + '/training_set' # training_set contains training dataset
    # test_dir = data_dir + '/test_set'  #contains test dataset
    train_dir = '/content/drive/MyDrive/Tree and Grass Information/train'
    test_dir = '/content/drive/MyDrive/Tree and Grass Information/test'

# Defining transformations for training and test data
# transforms.compose() will apply transformation to images
    transformation = transforms.Compose([transforms.RandomHorizontalFlip(),
                                         transforms.RandomRotation(20),
                                         transforms.Resize(size=(224, 224)),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                         ])
    classes = sorted(os.listdir(train_dir))
    #print(len(classes))
    # Load the dataset with Image Folder
    trainset = datasets.ImageFolder(train_dir, transform=transformation)
    #print(trainset.targets)
    testset = datasets.ImageFolder(test_dir, transform=transformation)

    # define data loaders
    batch_size = 32
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(testset, batch_size=batch_size, num_workers=1)

    # def imshow(img):
    #     img = img / 2 + 0.5  # unnormalize
    #     plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image
    #     #plt.show()
    # # obtain one batch of training images
    # data_iter = iter(train_loader)
    # images, lbls = data_iter.next()
    # images = images.numpy() # convert images to numpy for display
    # # plot the images in the batch, along with the corresponding labels
    # fig = plt.figure(figsize=(10, 4))
    # # display 20 images
    # for idx in np.arange(10):
    #     ax = fig.add_subplot(2, 10/2, idx+1, xticks=[], yticks=[])
    #     imshow(images[idx])
    #     label = lbls[idx]
    #     #ax.set_title(classes[label])
    #     #ax.set_title(classes[lbls[idx]])
    # Creating CNN classifier
    train_on_gpu = torch.cuda.is_available()  # check if Cuda is available for training


    # Initializing Parameters
    class Net(nn.Module):

        def __init__(self):
            super(Net, self).__init__()
            # convolutional layer1
            self.conv1 = nn.Conv2d(3, 16, 5)
            # max pooling layer
            self.pool = nn.MaxPool2d(2, 2)
            # convolutional layer2
            self.conv2 = nn.Conv2d(16, 32, 5)
            self.dropout = nn.Dropout(0.2)
            # Fully connected layer1
            self.fc1 = nn.Linear(32 * 53 * 53, 256)
            # fully connected layer2
            self.fc2 = nn.Linear(256, 84)
            # fully connected layer3
            self.fc3 = nn.Linear(84, len(classes))
            # Applying softmax function
            self.softmax = nn.LogSoftmax(dim=1)
            # feed forward network

        def forward(self, x):
            # add sequence of convolutional and max pooling layers
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.dropout(x)
            x = x.view(-1, 32 * 53 * 53)
            x = F.relu(self.fc1(x))
            x = self.dropout(F.relu(self.fc2(x)))
            x = self.softmax(self.fc3(x))
            return x

            # create Model instance


    model = Net()
    print(model)

    # move tensors to GPU if CUDA is available
    if (train_on_gpu):
        model.cuda()
        print("CUDA available")
    # specify loss function
    criterion = torch.nn.CrossEntropyLoss()
    # specify optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
    # Train Model
    # number of epochs to train the model
    n_epochs = 20  # you may increase this number to train a final model

    # valid_loss_min = np.Inf # track change in validation loss
    train_acc= []
    loss_values = []
    correct = 0
    total = 0
    for epoch in range(1, n_epochs + 1):

        # keep track of training and validation loss

        train_loss = 0.0
        #valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item() * data.size(0)
            # print statistics
            _, pred = torch.max(output, dim=1)
            correct += torch.sum(pred == target).item()
            total += target.size(0)
        # calculate average losses
        train_loss = train_loss / len(train_loader.dataset)
        loss_values.append(train_loss)
        train_acc.append(100 * correct / total)
        # print training/validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, train_loss))
    print("This is accuracy",train_acc)
    fig = plt.figure(figsize=(20, 10))
    plt.title("Train - Validation Loss")
    plt.plot(loss_values, label='train')
    #plt.plot(val_loss, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('loss', fontsize=12)
    plt.legend(loc='best')
    plt.show()

    fig = plt.figure(figsize=(20, 10))
    plt.title("Train - Validation Accuracy")
    plt.plot(train_acc, label='train')
    #plt.plot(val_acc, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('accuracy', fontsize=12)
    plt.legend(loc='best')
    plt.show()

    # Device configuration
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # Test the model
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)

            correct += (predicted == labels).sum().item()
        print(correct)
        print(total)
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.pth')

    network = Net()
    network.load_state_dict(torch.load('model.pth'))

    img = Image.open("/content/drive/MyDrive/Tree and Grass Information/test/Ashwagandha_test_Arman/1 (1).jfif")
    img_t = transformation(img)
    batch_t = torch.unsqueeze(img_t, 0)
    print("this is shape", batch_t.shape)
    network.eval()
    out = network(batch_t)
    print(out.shape)
    prediction = int(torch.max(out.data, 1)[1].numpy())
    print(prediction)
    print("this is class",classes[prediction])
    # if prediction == 0:
    #     print('limdo')
    # if prediction == 1:
    #     print('piplo')
    # if prediction == 2:
    #     print('vad')

FileNotFoundError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
